# Validators

ObsPlus provides a simple method for declaring and enforcing assumptions about data. You can think of it much like [pytest](https://docs.pytest.org/en/latest/) for data validation (i.e. it is enforced at runtime rather than in a test suite). The implementation is specifically geared towards nested tree structures (like obspy's `Catalog` object), but does work for any type of object. 

<div class="alert alert-warning">

**Warning**: This is a fairly advanced feature of ObsPlus intended primarily for library authors and users with stringent data requirements. The built-in validators will meet most people's needs.  
</div>

<div class="alert alert-warning">

**Warning**: In the future we may move much of this functionality to ObsPy as described in [this proposal](https://github.com/obspy/obspy/issues/2154), but an appropriate deprecation cycle will be implemented.
</div>


## Built-in Validators
Obsplus comes with a few built-in validators. See the [catalog validation](catalog_validation.ipynb) page for more details.

## Custom Validators
Here is an example which creates a custom validator for ensuring a group of events all have at least four picks and that the origins have latitude and longitude defined. We will use the namespace `"_silly_test"` to let obsplus know these validators should be grouped together.  


In [1]:
import obsplus
import obspy
import obspy.core.event as ev
from obsplus.utils.validate import validator, validate

namespace = '_silly_test'

In [2]:
# We simply have to decorate a callable with the `validator` decorator
# and specify the class it is to act on and its namespace
@validator(namespace, ev.Event)
def ensure_events_have_four_picks(event):
    picks = event.picks
    assert len(picks) >= 4

    
@validator(namespace, ev.Origin)
def ensure_origin_have_lat_lon(origin):
    assert origin.latitude is not None
    assert origin.longitude is not None
    

Now we create an event we know will violate both conditions and run the `validate` function. it should raise an `AssertionError`.

In [3]:
cat = obspy.read_events()

cat[0].picks = []

for origin in cat[0].origins:
    origin.latitude = None
    origin.longitude = None

In [4]:
try:
    validate(cat, namespace)
except AssertionError:
    print('catalog failed validations')

catalog failed validations


But we could also return a report of failures (in the form of a dataframe) if we don't want execution to stop, but rather what might be wrong with the data.

In [5]:
report = validate(cat, namespace, report=True)
report

,message,object,passed,validator
0,validator ensure_events_have_four_picks failed...,"[resource_id, event_type, event_type_certainty...",False,ensure_events_have_four_picks
1,validator ensure_events_have_four_picks failed...,"[resource_id, event_type, event_type_certainty...",False,ensure_events_have_four_picks
2,validator ensure_events_have_four_picks failed...,"[resource_id, event_type, event_type_certainty...",False,ensure_events_have_four_picks
3,validator ensure_origin_have_lat_lon failed ob...,"[resource_id, time, time_errors, longitude, lo...",False,ensure_origin_have_lat_lon
4,,"[resource_id, time, time_errors, longitude, lo...",True,ensure_origin_have_lat_lon
5,,"[resource_id, time, time_errors, longitude, lo...",True,ensure_origin_have_lat_lon


Notice how the `object` column is a reference to the python object which which the validator ran on. This makes is very quick to find (and fix) problematic data.

### Validators with optional arguments
You can also create validators that take optional arguments (in the form of key word arguments). The `validate` function then knows how to distribute these values to the appropriate validators.

In [6]:
@validator(namespace, ev.Origin)
def ensure_lat_greater_than(origin, min_lat=None):
    if min_lat is not None:
        print(f"min latitude is {min_lat}")
        assert origin.latitude is None or origin.latitude > min_lat

In [7]:
_ = validate(cat, namespace, min_lat=39, report=True)

min latitude is 39
min latitude is 39
min latitude is 39
